# Maizeを利用してSBVSの前処理とドッキングシミュレーションを実行する　その２
- maizeはよく利用されるワークフローはサブグラフとしてまとめて提供されています。このノートブックでは06と同じDockingのワークフローをmaizeで実装されているクラスから作成してみます。
- maize.graphs.mai以下にはDocking, FEP，Proteinprep，AutomaticSBDDのメソッドが定義されています。Proteinprep&&AutomaticSBDDはschrodinger社の製品用なので、ライセンスを持たれている方はチェックしてみるのもよいかと思います。

In [ ]:
#import setenv # XDG_CONFIG_HOMEの設定がうまく行かない場合
from rdkit import Chem
from maize.graphs.mai.dock import Docking
from maize.core.workflow import Workflow
from maize.steps.io import LoadData, Return
from maize.utilities.chem.chem import IsomerCollection
from pathlib import Path

In [2]:
flow = Workflow(name='docktest2')

- ここでは先の例と同じくSmilesのリストを受け取りそれをDockingノードに渡すというワークフローを作成します。
- Dockingサブグラフはソースコードを見ていただくとわかりますがGypsumによるリガンドプレパレーションとVinaによるDockingを実施するワークフローを定義しています。

In [3]:
load = flow.add(LoadData[list[str]])
dock = flow.add(Docking)
res = flow.add(Return[list[IsomerCollection]])

- DockingクラスはGypsumとVinaというノードから構成されています。Dockingのメソッドはn_variants, ph_rangeというGypsumの属性とreceptor, search_center, search_range, n_jobs, n_posesというVinaの属性は定義できますがexhaustivenessは陽に定義できません。
- dock.nodes['vina']とするとこでSubgraph内のVinaの属性のexhaustivenessを制御できます。
- Dockingの代わりにGlideDockingをインポートすればGlideDockingのワークフローも構築可能です（要ライセンス）

In [4]:
load.data.set(["COc1cc2c(cc1OCCCN3CCOCC3)c(ncn2)Nc4ccc(c(c4)Cl)F", 
               "COc1cc2c(cc1OCCCN3CCOCC3)c(ncn2)Nc4ccc(c(c4)Cl)"])
dock.n_variants = 1
dock.search_center.set((-56.643,  -6.844, -23.487))
dock.search_range.set((20.0, 20.0, 20.0))
dock.receptor.set(Path('./data/2ito_fix_remh.pdbqt'))
dock.nodes['vina'].exhaustiveness.set(16)

flow.connect(load.out, dock.inp)
flow.connect(dock.out, res.inp)

In [5]:
dock.parameters

{'n_variants': MultiParameter[<class 'int'>](name='n_variants', parent='docking'),
 'ph_range': MultiParameter[tuple[float, float]](name='ph_range', parent='docking'),
 'receptor': MultiParameter[typing.Annotated[pathlib.Path, Suffix(pdbqt)]](name='receptor', parent='docking'),
 'search_center': MultiParameter[tuple[float, float, float]](name='search_center', parent='docking'),
 'search_range': MultiParameter[tuple[float, float, float]](name='search_range', parent='docking'),
 'n_jobs': MultiParameter[<class 'int'>](name='n_jobs', parent='docking'),
 'n_poses': MultiParameter[<class 'int'>](name='n_poses', parent='docking')}

In [6]:
flow.execute()

2024-10-20 20:24:18,203 |     INFO |        docktest2 | 
              ___           ___                       ___           ___
             /\__\         /\  \          ___        /\  \         /\  \
            /::|  |       /::\  \        /\  \       \:\  \       /::\  \
           /:|:|  |      /:/\:\  \       \:\  \       \:\  \     /:/\:\  \
          /:/|:|__|__   /::\~\:\  \      /::\__\       \:\  \   /::\~\:\  \
         /:/ |::::\__\ /:/\:\ \:\__\  __/:/\/__/ _______\:\__\ /:/\:\ \:\__\
         \/__/~~/:/  / \/__\:\/:/  / /\/:/  /    \::::::::/__/ \:\~\:\ \/__/
               /:/  /       \::/  /  \::/__/      \:\~~\~~      \:\ \:\__\
              /:/  /        /:/  /    \:\__\       \:\  \        \:\ \/__/
             /:/  /        /:/  /      \/__/        \:\__\        \:\__\
             \/__/         \/__/                     \/__/         \/__/


2024-10-20 20:24:18,203 |     INFO |        docktest2 | Starting Maize version 0.8.3 (c) AstraZeneca 2024
2024-10-20 20:2

2024-10-20 20:26:18,835 |     INFO |             vina | Job completed (1/2)


2024-10-20 20:26:33,558 |     INFO |             vina | Job completed (2/2)
2024-10-20 20:26:33,649 |     INFO |        docktest2 | Workflow status
                                   |           gypsum | COMPLETED
                                   |           return | WAITING_FOR_INPUT
                                   |         loaddata | COMPLETED
                                   |             vina | RUNNING
2024-10-20 20:26:33,648 |     INFO |             vina | Parsing isomer 0: 'XGALLCVXEZPNRQ-LELJVTLKNA-N'
2024-10-20 20:26:33,651 |     INFO |             vina | Parsing isomer 1: 'VSXSFJNZEWFVCZ-HXTKINSTNA-N'
2024-10-20 20:26:33,657 |     INFO |        docktest2 | Node 'return' finished (3/4)


2024-10-20 20:26:34,155 |     INFO |        docktest2 | Node 'vina' finished (4/4)
2024-10-20 20:26:34,657 |     INFO |        docktest2 | Execution completed :) total runtime: 0:02:15.930356
	4 nodes completed successfully
	0 nodes stopped due to closing ports
	0 nodes failed
	0:04:32.069589 total walltime
	0:02:17.322131 spent waiting for resources or other nodes
